In [9]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv('../output/results.csv', engine='pyarrow')
df = df[df['tech'] != 'm1'] # This is merging files on the laptop
df.head()

,time,function,tech,merged,filename,step,file_mb,row_count,sent_mb,recv_mb,sent_mb_1s,recv_mb_1s,error,data_dir,file_count,datetime,id
0,0.040757,merge_files,m1,True,0.parquet,1,114.833744,10000000,0.0,0.000000,0.017578,0.008789,None,mock,10,01/08/2023 17:49:53,-2655337975328636541
1,0.053151,copy_file,m1,False,0.parquet,1,114.833744,10000000,0.0,0.000000,0.008789,0.008789,None,mock,10,01/08/2023 17:49:53,-2655337975328636541
2,0.032379,copy_file,m1,True,0.parquet,1,114.833744,10000000,0.0,0.000000,0.010742,0.009766,None,mock,10,01/08/2023 17:49:53,-2655337975328636541
3,0.055717,copy_file,m1,False,0.parquet,1,114.833744,10000000,0.0,0.000000,0.002930,0.004883,None,mock,10,01/08/2023 17:49:53,-2655337975328636541
4,0.029708,copy_file,m1,True,0.parquet,1,114.833744,10000000,0.0,0.000977,0.006836,0.008789,None,mock,10,01/08/2023 17:49:53,-2655337975328636541


In [21]:
groups = df.groupby(['tech','merged'])

fig = go.Figure()
for name, group in groups:
    fig.add_trace(go.Scatter(x=group['step'], y=group['time'], mode='lines', name=str(name)))
fig.update_layout(
    xaxis_title='Step',
    yaxis_title='Time'
)
fig.show()